In [1]:
from __future__ import print_function
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity='all'

from matplotlib import pyplot as plt
%pylab inline

# %load_ext autoreload
# %autoreload 2
import numpy as np
import pandas as pd 
pd.options.display.max_rows=5  # 设置dataframe最多的显示行数
import os
import sys
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())
    print(f"After inserting, sys.path is {sys.path}")

Populating the interactive namespace from numpy and matplotlib
After inserting, sys.path is ['/home/hesy/.conda/envs/py36/lib/python36.zip', '/home/hesy/.conda/envs/py36/lib/python3.6', '/home/hesy/.conda/envs/py36/lib/python3.6/lib-dynload', '', '/home/hesy/.conda/envs/py36/lib/python3.6/site-packages', '/home/hesy/env/gym', '/home/hesy/.conda/envs/py36/lib/python3.6/site-packages/IPython/extensions', '/tmp/tmpcz8vy2y8', '/home/hesy/pro/team-learning-rs/RecommandNews']


# questions

<div class="alert alert-block alert-warning">
<b>Questions to be solved</b> 
    
    * data_sample = pd.read_csv('data/sample_submit.csv') , 取消index ,使用index_col=None? 应该不是
    * jupyter lab好的插件：1. 注释高亮（like better comments in vsCodes） 2. title folding(是不是我的失效了...)
</div>

# data observation

In [13]:
data_sample = pd.read_csv('data/sample_submit.csv')
data_articles = pd.read_csv('data/articles.csv')
data_articles_emb = pd.read_csv('data/articles_emb.csv')
data_train_click = pd.read_csv('data/train_click_log.csv')
data_testA_click = pd.read_csv('data/testA_click_log.csv')

In [46]:
data_sample.shape
data_articles.shape
data_articles_emb.shape
f"(training data size/data test) size is {data_train_click.size/data_testA_click.size}"

(50000, 6)

(364047, 4)

(364047, 251)

'(training data size/data test) size is 2.1478793845678656'

In [41]:
# print(list(filter(lambda x : x if '_' not in x else None ,dir(data_train_click) )),end=",\t")
data_train_click.columns

Index(['user_id', 'click_article_id', 'click_timestamp', 'click_environment',
       'click_deviceGroup', 'click_os', 'click_country', 'click_region',
       'click_referrer_type'],
      dtype='object')

In [48]:
print("===== training data =====")
data_train_click.shape
len(set(data_train_click.user_id))
len(set(data_train_click.click_article_id))

print("===== testing data =====")
data_testA_click.shape
len(set(data_testA_click.user_id))
len(set(data_testA_click.click_article_id))

===== training data =====


(1112623, 9)

200000

31116

===== testing data =====


(518010, 9)

50000

16330

训练集 采了100w条数据，20w个用户，3w个目标article； 测试集A 采51w条数据，5w个用户，1.6w个目标article； 测试集B是另外5w个用户

但是号称有36w的article，也就是大多数的标签都是0啊？

In [11]:
print("===== data_sample =====");data_sample.head(3)
print("===== data_articles =====");data_articles.head(3)
print("===== data_articles_emb =====");data_articles_emb.head(3)
print("===== data_train_click =====");data_train_click.head(3)
print("===== data_testA_click =====");data_testA_click.head(3)

===== data_sample =====


,user_id,article_1,article_2,article_3,article_4,article_5
0,200000,1,2,3,4,5
1,200001,1,2,3,4,5
2,200002,1,2,3,4,5


===== data_articles =====


,article_id,category_id,created_at_ts,words_count
0,0,0,1513144419000,168
1,1,1,1405341936000,189
2,2,1,1408667706000,250


===== data_articles_emb =====


,article_id,emb_0,emb_1,emb_2,emb_3,emb_4,emb_5,emb_6,emb_7,emb_8,...,emb_240,emb_241,emb_242,emb_243,emb_244,emb_245,emb_246,emb_247,emb_248,emb_249
0,0,-0.161183,-0.957233,-0.137944,0.050855,0.830055,0.901365,-0.335148,-0.559561,-0.500603,...,0.321248,0.313999,0.636412,0.169179,0.540524,-0.813182,0.286870,-0.231686,0.597416,0.409623
1,1,-0.523216,-0.974058,0.738608,0.155234,0.626294,0.485297,-0.715657,-0.897996,-0.359747,...,-0.487843,0.823124,0.412688,-0.338654,0.320786,0.588643,-0.594137,0.182828,0.397090,-0.834364
2,2,-0.619619,-0.972960,-0.207360,-0.128861,0.044748,-0.387535,-0.730477,-0.066126,-0.754899,...,0.454756,0.473184,0.377866,-0.863887,-0.383365,0.137721,-0.810877,-0.447580,0.805932,-0.285284


===== data_testA_click =====


,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,249999,160974,1506959142820,4,1,17,1,13,2
1,249999,160417,1506959172820,4,1,17,1,13,2
2,249998,160974,1506959056066,4,1,12,1,13,2


===== data_train_click =====


,user_id,click_article_id,click_timestamp,click_environment,click_deviceGroup,click_os,click_country,click_region,click_referrer_type
0,199999,160417,1507029570190,4,1,17,1,13,1
1,199999,5408,1507029571478,4,1,17,1,13,1
2,199999,50823,1507029601478,4,1,17,1,13,1
